In [1]:
import logging

import nemo.collections.asr as nemo_asr
import pandas as pd
import torch
from nemo.collections.nlp.models import PunctuationCapitalizationModel
from pyannote.audio import Pipeline
from pydub import AudioSegment
from pydub.silence import detect_silence
from ctcdecode import CTCBeamDecoder

logging.getLogger("nemo_logger").setLevel(logging.ERROR)
asr_logger = logging.getLogger("asr")
asr_logger.setLevel(logging.INFO)


[NeMo W 2022-09-01 12:59:58 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-09-01 13:00:00 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-09-01 13:00:00 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


## Model spot-testing

In [2]:
pretrained_models = [e for e in nemo_asr.models.ASRModel.list_available_models() if 'stt_en' in e.pretrained_model_name]

In [ ]:
# one of each? https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/models.html
test_models = ['stt_en_quartznet15x5', 'stt_en_citrinet_512', 'stt_en_contextnet_512', 'stt_en_conformer_ctc_medium', 'stt_en_conformer_transducer_medium']

In [ ]:
['stt_en_citrinet_1024',
 'stt_en_citrinet_1024_gamma_0_25',
 'stt_en_citrinet_256',
 'stt_en_citrinet_256_gamma_0_25',
 'stt_en_citrinet_512',
 'stt_en_citrinet_512_gamma_0_25',
 'stt_en_conformer_ctc_large',
 'stt_en_conformer_ctc_large_ls',
 'stt_en_conformer_ctc_medium',
 'stt_en_conformer_ctc_medium_ls',
 'stt_en_conformer_ctc_small',
 'stt_en_conformer_ctc_small_ls',
 'stt_en_conformer_ctc_xlarge',
 'stt_en_conformer_transducer_large',
 'stt_en_conformer_transducer_large_ls',
 'stt_en_conformer_transducer_medium',
 'stt_en_conformer_transducer_small',
 'stt_en_conformer_transducer_xlarge',
 'stt_en_conformer_transducer_xxlarge',
 'stt_en_contextnet_1024',
 'stt_en_contextnet_1024_mls',
 'stt_en_contextnet_256',
 'stt_en_contextnet_256_mls',
 'stt_en_contextnet_512',
 'stt_en_contextnet_512_mls',
 'stt_en_jasper10x5dr',
 'stt_en_quartznet15x5',
 'stt_enes_conformer_ctc_large',
 'stt_enes_conformer_transducer_large',
 'stt_enes_contextnet_large']

In [4]:
[e.pretrained_model_name for e in pretrained_models]

['stt_en_citrinet_1024',
 'stt_en_citrinet_1024_gamma_0_25',
 'stt_en_citrinet_256',
 'stt_en_citrinet_256_gamma_0_25',
 'stt_en_citrinet_512',
 'stt_en_citrinet_512_gamma_0_25',
 'stt_en_conformer_ctc_large',
 'stt_en_conformer_ctc_large_ls',
 'stt_en_conformer_ctc_medium',
 'stt_en_conformer_ctc_medium_ls',
 'stt_en_conformer_ctc_small',
 'stt_en_conformer_ctc_small_ls',
 'stt_en_conformer_ctc_xlarge',
 'stt_en_conformer_transducer_large',
 'stt_en_conformer_transducer_large_ls',
 'stt_en_conformer_transducer_medium',
 'stt_en_conformer_transducer_small',
 'stt_en_conformer_transducer_xlarge',
 'stt_en_conformer_transducer_xxlarge',
 'stt_en_contextnet_1024',
 'stt_en_contextnet_1024_mls',
 'stt_en_contextnet_256',
 'stt_en_contextnet_256_mls',
 'stt_en_contextnet_512',
 'stt_en_contextnet_512_mls',
 'stt_en_jasper10x5dr',
 'stt_en_quartznet15x5',
 'stt_enes_conformer_ctc_large',
 'stt_enes_conformer_transducer_large',
 'stt_enes_contextnet_large']

In [32]:
pretrained_models[0].class_.from_pretrained(model_name=pretrained_models[0].pretrained_model_name)

In [2]:
BATCH_SIZE = 1
MS = 1000
model_name = 'stt_en_conformer_ctc_small'
model = nemo_asr.models.ASRModel.from_pretrained(model_name=model_name)

In [31]:
del model
torch.cuda.empty_cache()

In [5]:
import tempfile
import nvsmi
from pathlib import Path
import time
import torch

model_memory_footprint = nvsmi.get_gpu_processes()[0].used_memory
audio_segment = AudioSegment.from_wav('/home/blog-os-asr/output/temp_dir/rewilding-the-scottish-highlands.wav')
slice_intervals = [(0, e*MS) for e in list(range(0,135,15))][1:]

memory_usage_records = []
with tempfile.TemporaryDirectory() as temp_dir:
    for interval in slice_intervals:
        save_message = audio_segment[interval[0]:interval[1]].export(Path(temp_dir) / 'memory_test_fragment.wav', format='wav')
        (Path(temp_dir) / 'memory_test_fragment.wav').exists()
        [str(Path(temp_dir) / 'memory_test_fragment.wav')]
        before = time.time()
        transcription = model.transcribe(paths2audio_files=[str(Path(temp_dir) / 'memory_test_fragment.wav')], batch_size=BATCH_SIZE)
        after = time.time()
        memory_usage_records.append({'transcript': transcription, 'memory_usage': nvsmi.get_gpu_processes()[0].used_memory, 'time_elapsed': after-before})


True

['/tmp/tmpcs6gqizs/memory_test_fragment.wav']

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

True

['/tmp/tmpcs6gqizs/memory_test_fragment.wav']

...

['/tmp/tmpcs6gqizs/memory_test_fragment.wav']

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

True

['/tmp/tmpcs6gqizs/memory_test_fragment.wav']

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

## Test script

In [75]:
transcription

["but we begin with jeremy leggate former professor at imperial college and oxford with his massive project in a world famous setting highland's rewlding is a new type of company it's a mass ownership company and it's going to do its level best to accelerate nature recovery in the highlands of scotland so as to contribute to repair of climate meltdown and biodiversity collapse thats some remits't it jeremy legget could you place us where are we i know down below is lone but loness much to my surprise is very very long so where are we we' are halfway along lockness about sixteen miles from a fairly big city invenness the capital of the highlands of scotland and despite the proximity to that city we're in a a really wild place there's one thousand two hundred acres on this estate that we're managing and not many footpaths and there's a lot of genuine wildland in our mixed habitats of native woodlands peedlands pasture heathland and conniferous plantations how long is lochnes"]

In [ ]:
from asr import transcribe_mono

transcription = transcribe_mono(
    "../output/radio_national_podcasts/audio/rewilding-the-scottish-highlands.mp3"
)
transcription

In [ ]:
time_formatted_words_all = []
for idx, record in transcription.head(10).iterrows():
    time_formatted_words = _format_word_timestamps(record.asr_outputs, record.start)

    # 5.0 apply punctuation to each output
    punctuated_sequence = punct_model.add_punctuation_capitalization(
        [" ".join(e["word"] for e in time_formatted_words)]
    )[0]

    # if len(punctuated_sequence.split(" ")) == len(time_formatted_words):
    #     # easy case, where punctuated output len matches input len; assign directly
    #     punctuated_sequence_joined = (
    #         pd.DataFrame(time_formatted_words)
    #         .assign(word=punctuated_sequence.split(" "))
    #         .assign(speakerTag=record.speaker)
    #         .to_dict(orient="records")
    #     )
    #     time_formatted_words_all.append(punctuated_sequence_joined)
    # else:
    #     # otherwise.. pad the difference? changes should be limited to immediately proceeding fullstops, commas, question marks
    #     # https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html
    #     print("Punctuated outputs not the same length as input")